# CS505 Final Project
#### A solo project by Arthur Savage - U02936938

The premise for this project is to 

Do some stuuuuufffffff


In [4]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.1 MB/s            
     |████████████████████████████████| 880 kB 102.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.6 MB 98.6 MB/s            
     |████████████████████████████████| 759 kB 96.6 MB/s            


     |████████████████████████████████| 97 kB 93.6 MB/s            
     |████████████████████████████████| 309 kB 97.3 MB/s            
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=6cda933646b9882201eb28da7879e0aa292c32f6d516dbd08126d2a3160d654e
  Stored in directory: /scratch/3154247.1.ece/pip-ephem-wheel-cache-nfclx_zg/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
Successfully built sacremoses


In [7]:
import transformers
from transformers import pipeline
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from datasets import load_dataset, list_datasets
from tqdm import tqdm
import pandas as pd


import torch
import torch.nn.functional as F
from torch.utils.data import random_split,Dataset,DataLoader
from torch import nn, optim


ModuleNotFoundError: No module named 'huggingface'

In [ ]:
from datasets import Dataset
Dataset.cleanup_cache_files

In [ ]:
generator = pipeline('text-generation', model='gpt2', device=0)
outputs = generator('I wonder what I will generate?')
print(outputs)

In [ ]:
# initializing GPT-2 model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2', device=0)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# load and tokenize dataset
def encode(batch): return tokenizer([x.strip('@user') for x in batch['tweet']], truncation=True, padding=True)

fake_news = load_dataset('tweets_hate_speech_detection', split='train')
processed = fake_news.map(encode, batched=True)
processed.set_format('torch', columns=['input_ids', 'attention_mask'])


In [ ]:
# initializing trainer
training_args = TrainingArguments(
    output_dir='/projectnb/cs505ws/students/savageaf/project/fake_news/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,                 # gpt-2
    tokenizer=tokenizer, 
    args=training_args,          # 
    data_collator=data_collator, # tells Trainer not to look for vector of labels
    train_dataset=processed
)

In [ ]:
# fine-tune model on hate speech tweets]
torch.cuda.empty_cache()
#torch.cuda.set_max_workspace_size(max_split_size_mb * 1024 * 1024)

trainer.train()

In [ ]:
trainer.save_model('./hate_tweet_model')
hate_tweets = pipeline('text-generation', model='./hate_tweet_model', device=0)
regular_gpt2 = pipeline('text-generation', model='gpt2', device=0)

In [ ]:
# generating text and comparing models
print("REGULAR GPT-2:")
print(regular_gpt2('It makes me mad when'))

print('\n')

print("HATE TWEETS MODEL:")
print(hate_tweets('It makes me mad when'))

### NOW WE TRY MODEL TWO

iT DOSE oTHEer stuff

In [ ]:
def encode(batch): return tokenizer([x for x in batch['text']], truncation=True, padding=True)
from datasets import Dataset

df = pd.read_csv('/project/perchash/WS/project/Fake.csv')
fake_news = Dataset.from_pandas(df)
processed = fake_news.map(encode, batched=True)
processed.set_format('torch', columns=['input_ids', 'attention_mask'])

In [ ]:
# initializing second trainer
training_args2 = TrainingArguments(
    output_dir='/projectnb/cs505ws/students/savageaf/project/fake_news2/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./logs'
)

trainer2 = Trainer(
    model=model,                 # gpt-2
    tokenizer=tokenizer, 
    args=training_args2,          # 
    data_collator=data_collator, # tells Trainer not to look for vector of labels
    train_dataset=processed
)

In [ ]:
torch.cuda.empty_cache()
#torch.cuda.set_max_workspace_size(max_split_size_mb * 1024 * 1024)

trainer2.train()

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
     def __init__(self, data: list[list[int]]):
         self.data = []
         for d in data:
             input_ids = torch.tensor(d, dtype=torch.int64)
             attention_mask = torch.ones(len(d), dtype=torch.int64)
             self.data.append({'input_ids': input_ids,
                  'attention_mask': attention_mask, 'labels': input_ids})
 
     def __len__(self):
         return len(self.data)
 
     def __getitem__(self, idx: int):
         return self.data[idx]


def train_one(model, loader, optimizer):
     model.train()
     losses = []
     for batch in tqdm.tqdm(loader):
         for k, v in batch.items():
             batch[k] = v.to(DEVICE)
         optimizer.zero_grad()
         out = model(input_ids=batch['input_ids'],
                     attention_mask=batch['attention_mask'],
                     labels=batch['labels'])
         loss = out['loss']
         loss.backward()
         optimizer.step()
         losses.append(loss.item())

     return np.mean(losses)

In [ ]:
batch_size = 64

# calling Dataset constructor and splitting dataset
full_dataset = CustomDataset(X,Y)
training_dataset, validation_dataset, testing_dataset = random_split(full_dataset, [0.7,0.1,0.2])

# creating DataLoader objects of the training, validation, and testing datasets
training_dataloader =   DataLoader(training_dataset, batch_size=batch_size, shuffle=False)
validation_dataloader = DataLoader(validation_dataset,   batch_size=batch_size, shuffle=False)
testing_dataloader =    DataLoader(testing_dataset,  batch_size=batch_size, shuffle=False)

n_train = len(training_dataloader.dataset)
n_val = len(validation_dataloader.dataset)
n_test = len(testing_dataloader.dataset)

In [ ]:

loader_train = torch.utils.data.DataLoader(training_dataset, batch_size=1)
loader_val = torch.utils.data.DataLoader(validation_dataset, batch_size=1)

DEVICE = 'cuda'    # or 'cpu'
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for i_epoch in range(20):
     loss_train = train_one(model, loader_train, optimizer)
     loss_val = val_one(model, loader_val)
     print(f'{i_epoch} : loss_train={loss_train}, loss_val={loss_val}')




In [ ]:
import os
import time
import datetime
#from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
#% matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

In [ ]:
filename = '/project/perchash/WS/project/Fake.csv' # fake news CSV from Kaggle
df = pd.read_csv(filename) # open CSV
df = df['text'] # only select text

sentences = []
for index in df.index: # iterate through each row of dataframe
    document = df[index] # go document by document
    split_document = document.split('.') # splits sentences of document by period
    sentences = sentences + split_document # concatenates all sentences into a single list of sentences

sentences = sentences[:2000] # cutting down dataset size so I finish on time

In [ ]:
# taken from HW6
doc_lengths = []

for doc in sentences:

    # get rough token count distribution
    tokens = nltk.word_tokenize(doc)

    doc_lengths.append(len(tokens))

doc_lengths = np.array(doc_lengths)

sns.distplot(doc_lengths)

In [ ]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
batch_size = 2


In [ ]:
dataset = GPT2Dataset(sentences, tokenizer, max_length=768)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# some parameters I cooked up that work reasonably well

epochs = 4
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [ ]:
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 200,
                                    top_p=0.95,
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
#                            token_type_ids=None,
                             attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
# Display floats with two decimal places.
#pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


In [ ]:
model.eval()

prompt = "<|startoftext|>"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated,
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,
                                top_k=50,
                                max_length = 300,
                                top_p=0.95,
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

### Sources:
1. https://www.it-jim.com/blog/training-and-fine-tuning-gpt-2-and-gpt-3-models-using-hugging-face-transformers-and-openai-api/
2. https://medium.com/@pierre_guillou/faster-than-training-from-scratch-fine-tuning-the-english-gpt-2-in-any-language-with-hugging-f2ec05c98787
3. https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/
4. 